# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt 


# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vernon,50.2581,-119.2691,2.64,100,100,0.51,CA,1707254254
1,1,blackmans bay,-43.0167,147.3167,14.07,74,97,2.61,AU,1707254396
2,2,hermanus,-34.4187,19.2345,19.89,79,47,6.87,ZA,1707254234
3,3,grytviken,-54.2811,-36.5092,9.82,57,2,1.92,GS,1707254194
4,4,palmas,-10.2128,-48.3603,27.93,78,20,1.54,BR,1707254082


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
"Lng",
"Lat",
geo=True,
color="City",
size="Humidity",
scale = .5,
alpha=.5,
tiles="OSM")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
vacayDf = city_data_df.loc[city_data_df["Wind Speed"] <= 5]
vacayDf = vacayDf.loc[vacayDf["Cloudiness"]==0]
vacayDf = vacayDf.loc[vacayDf["Max Temp"]>=22]

# Drop any rows with null values
vacayDf.count()
vacayDf = vacayDf.dropna(how="any")
vacayDf.count()

# Display sample data
vacayDf.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
160,160,san julian,21.0167,-102.1667,23.82,9,0,2.19,MX,1707254413
167,167,illapel,-31.6308,-71.1653,27.15,50,0,3.86,CL,1707254172
169,169,siyabuswa,-25.1132,29.0445,24.76,31,0,1.10,ZA,1707254414
200,200,sao vicente,-23.9631,-46.3919,29.59,57,0,4.02,BR,1707254244
262,262,bopolu,7.0667,-10.4875,24.31,86,0,1.61,LR,1707254431
280,280,saint-pierre,-21.3393,55.4781,23.82,64,0,2.57,RE,1707254434
296,296,obo,5.3998,26.4869,23.51,23,0,2.56,CF,1707254435
351,351,gabu,12.2833,-14.2167,29.26,15,0,0.93,GW,1707254298
353,353,jiquilisco,13.3167,-88.5833,32.05,37,0,3.64,SV,1707254441
354,354,salalah,17.0151,54.0924,23.05,64,0,2.06,OM,1707254442


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacayDf[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

/var/folders/_9/lmr64h8x6pnd8t11zwp4xgpr0000gn/T/ipykernel_88587/1076565594.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
160,san julian,MX,21.0167,-102.1667,9,
167,illapel,CL,-31.6308,-71.1653,50,
169,siyabuswa,ZA,-25.1132,29.0445,31,
200,sao vicente,BR,-23.9631,-46.3919,57,
262,bopolu,LR,7.0667,-10.4875,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    'limit':20,
    "apiKey":{geoapify_key}   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f'proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san julian - nearest hotel: No hotel found
illapel - nearest hotel: Domingo Ortiz de Rosas
siyabuswa - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
bopolu - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
obo - nearest hotel: No hotel found
gabu - nearest hotel: HOTEL VISIOM
jiquilisco - nearest hotel: Motel Jardín
salalah - nearest hotel: Muscat International Hotel
anantapur - nearest hotel: Geetha Residency
pathein - nearest hotel: Taan Taan Ta
mopti - nearest hotel: No hotel found
sukuta - nearest hotel: London Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
160,san julian,MX,21.0167,-102.1667,9,No hotel found
167,illapel,CL,-31.6308,-71.1653,50,Domingo Ortiz de Rosas
169,siyabuswa,ZA,-25.1132,29.0445,31,No hotel found
200,sao vicente,BR,-23.9631,-46.3919,57,Pousada Vitória
262,bopolu,LR,7.0667,-10.4875,86,No hotel found
280,saint-pierre,RE,-21.3393,55.4781,64,Tropic Hotel
296,obo,CF,5.3998,26.4869,23,No hotel found
351,gabu,GW,12.2833,-14.2167,15,HOTEL VISIOM
353,jiquilisco,SV,13.3167,-88.5833,37,Motel Jardín
354,salalah,OM,17.0151,54.0924,64,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Hotel Name", "Country"],
    geo=True,
    color="City",
    size="Humidity",
    scale = .95,
    alpha=.95,
    tiles="OSM",
        frame_width = 800,
        frame_height = 600
    )
    
# Display the map
    

hotel_map

plt.savefig("output_data/Vac.png")

<Figure size 640x480 with 0 Axes>